# Тетрадь 6: Алгоритм k-ближайших соседей

***

## Теория

Алгоритм k-ближайших соседей KNN — это один из самых простых и интуитивно понятных алгоритмов классификации. Он относится к
методам **ленивого обучения** (lazy learning), так как не строит модель на этапе обучения, а
откладывает все вычисления до момента классификации нового объекта.

#### Основные шаги алгоритма

1. **Выбор числа соседей (k)**
   - Параметр k определяет, сколько ближайших соседей будет учитываться при классификации.
   - Обычно выбирается нечётное число, чтобы избежать ничьей при голосовании.

2. **Вычисление расстояний**
   - Для нового объекта вычисляются расстояния до всех объектов в обучающей выборке.
   - Часто используются метрики расстояния, такие как евклидово расстояние, манхэттенское расстояние
   или косинусное сходство.

3. **Выбор k ближайших соседей**
   - Из обучающей выборки выбираются $k$ объектов, которые находятся ближе всего к новому объекту.

4. **Голосование**
   - Класс нового объекта определяется путём голосования среди $k$ ближайших соседей.
   - Например, если $k=3$, и два соседа принадлежат к классу $A$, а один — к классу $B$, то новый
   объект будет отнесён к классу $A$.

5. **Возвращение результата**:
   - Алгоритм возвращает предсказанный класс для нового объекта.

#### Преимущества:

- Простота реализации и интерпретации.
- Не требует этапа обучения (все вычисления происходят на этапе предсказания).
- Хорошо работает на данных с небольшим количеством признаков.

#### Недостатки:

- Вычислительная сложность на больших данных, так как для каждого нового объекта нужно вычислять
расстояния до всех объектов в обучающей выборке.
- Чувствительность к масштабированию данных (необходимо нормализовать признаки).
- Чувствительность к выбору параметра $k$: слишком маленькое $k$ может привести к переобучению, а
слишком большое — к недообучению.

In [ ]:
from scipy.stats import mode


class LoopingKNNClassifier(ModelBase):
    # This K-Nearest Neighbors Classifier loops twice, comparing each train and
    # test point in the set. Uses Euclidean distance to determine the nearest
    # neighbors.

    def __init__(self):
        super().__init__()

    def fit(self, x_train: ndarray, y_train: ndarray, *args, **kwargs) -> None:
        super().fit(x_train, y_train)

    def predict(self, x_test, n_neighbors: int = 5):
        # This method is where the distances calculation happens.
        n_test_samples = x_test.shape[0]
        n_train_samples = x_train.shape[0]
        distances = np.zeros((n_test_samples, n_train_samples))

        # Iterating over each test sample,
        for i in range(n_test_samples):
            # iterate over each training sample,
            for j in range(n_train_samples):
                # calculate the Euclidean distance between the current test
                # sample (`x_test[i]`) and the current training sample
                # (`x_train[j]`).
                distances[i, j] = np.sqrt(
                    np.sum(np.square(x_test[i, :] - x_train[j, :]))
                )

        # Predict the lables based on the distances calculated.
        predicted_labels = self._predict_labels(distances, n_neighbors)
        return predicted_labels

    def _predict_labels(
        self,
        distances: ndarray,
        n_neighbors: int = 5,
    ) -> ndarray:
        # Predicting labels is a separate process. That's why it has it's own
        # private method.
        n_test_samples = distances.shape[0]
        y_predicted = np.zeros((n_test_samples, 1))

        # Iterating over each test sample to predict its label,
        for i in range(n_test_samples):
            # sort the distances for the current test sample and get the
            # indices of the sorted distances,
            sorted_indices = np.argsort(distances[i, :])
            # determine the number of neighbors to consider, ensuring it does
            # not exceed the available neighbors,
            farthest_neighbor = np.min([n_neighbors, len(sorted_indices)])
            # get the labels of the closest neighbors from the training set,
            closest_neighbors = self.y_train[
                sorted_indices[:farthest_neighbor]
            ]
            # use the mode (most frequent label) of the closest neighbors as
            # the predicted label.
            y_predicted[i] = mode(closest_neighbors)[0]
        return y_predicted

## Задание

***